<a href="https://colab.research.google.com/github/towardsai/ai-tutor-rag-system/blob/notebook%2Faman/notebooks/Basic_Agent_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install requirements

In [ ]:
pip install -U duckduckgo_search openai==1.51.0 --quiet

## Setup Enviornment variables

In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = "<OPENAI_API_KEY>"

Enter your OPENAI API key: ··········


## Setting Up Tools

In [ ]:
from duckduckgo_search import DDGS

def search(query):
    """
    perform a search on for the given query and return the results
    :param query: the query to search for
    :return: the search results
    """
    response = DDGS().text(query, max_results=5)
    results = [
        {"snippet": r["body"], "title": r["title"], "link": r["href"]}
        for r in response
    ]

    formatted_results = ""
    for result in results:
        formatted_results += f"Title: {result['title']}\n"
        formatted_results += f"Snippet: {result['snippet']}\n"
        formatted_results += "----\n"

    return formatted_results


In [ ]:
def evaluate_expression(operation: str) -> float:
    """
    perform a calculation on the given operation and return the result
    :param operation: the operation to perform, should be compatible to use with eval eg: operation: "1+2"
    :return: the result of the operation
    """
    return eval(operation)

In [ ]:
print(search("llama index"))

Title: LlamaIndex Newsletter 2024-10-22
Snippet: VividNode, a cross-platform desktop app that enables chatting with models like GPT, Claude, Gemini, and Llama. It features a chat interface, image generation capabilities with DALL-E 3, and advanced settings for privacy and convenience, all integrated with LlamaIndex. Github Repo, Tweet.
----
Title: Which Tools to Use for LLM-Powered Applications: LangChain vs ...
Snippet: In this section, you'll learn how to use LlamaIndex to create a queryable index from a collection of documents and interact with OpenAI's API for querying purposes. This is the code to do this: pip install llama-index %env OPENAI_API_KEY = "your-api-key" from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
----
Title: Unleashing the Power of Gemini With LlamaIndex - DZone
Snippet: pip install -q llama-index google-generativeai pip install llama-index-llms-gemini pip install llama-index-embeddings-gemini. Next, import the required modules from LlamaIndex

## Creating Agent

In [ ]:
import re
from openai import OpenAI

class Agent:
    def __init__(self, client: OpenAI, tools):
        """
        Initialize the Agent class.

        Args:
        client (OpenAI): The OpenAI client instance.
        tools (list): A list of tool functions.
        """
        self.client = client
        self.tools = tools
        self.memory = []

    def get_agent_prompt(self) -> str:
        """
        Generate the agent prompt string.

        Returns:
        str: The agent prompt string.
        """
        tools_str = "\n".join(
            [f"Action:{tool.__name__}\nAction Input:{tool.__doc__} \n" for tool in self.tools]
        )
        system_prompt = f"""
        You run in a loop of Thought, Action, Observation.
        At the end of the loop you output an Answer
        Use Thought to describe your thoughts about the question you have been asked.
        Use Action to run one of the actions available to you.
        Observation will be the result of running those actions.

        Your available actions are:
          {tools_str}

        Try to break the search query into multiple query for good results

        Always return in this format
        Question: the question
        Thought: your thoughts about the question
        Action: tool_name: the input to the action

        Example session:
        Question: New York is in which country?
        Thought: I should search for new york in web
        Action: search: new york

        You will be called again with this:

        Observation: New york is a city in the United States of America.

        You then output:
        Final Answer: New York is in the United States of America.
        """
        return system_prompt

    def add_agent_steps_in_memory(self, role: str, content: str) -> None:
        """
        Add agent steps to memory.

        Args:
        role (str): The role of the agent step.
        content (str): The content of the agent step.
        """
        self.memory.append({"role": role, "content": content})

    def invoke(self, user_input: str, max_iterations: int = 10) -> str:
        """
        Invoke the agent.

        Args:
        user_input (str): The user input string.
        max_iterations (int): The maximum number of iterations.

        Returns:
        str: The final answer.
        """
        i = 0
        self.add_agent_steps_in_memory("system", self.get_agent_prompt())
        current_message = user_input
        tool_names = [tool.__name__ for tool in self.tools]

        while i < max_iterations:
            print(f"Iteration: {i+1}")
            i += 1
            self.add_agent_steps_in_memory("user", current_message)
            result = self.client.chat.completions.create(
                messages=self.memory,
                model="gpt-4o",
                temperature=0.2
            )
            agent_output = result.choices[0].message.content
            print(agent_output)
            self.add_agent_steps_in_memory("assistant", agent_output)

            if "Action" in agent_output:
                action = re.findall(r"Action: ([a-z_]+): (.+)", agent_output, re.IGNORECASE)
                print(action)
                chosen_tool = action[0][0]
                arg = action[0][1]

                if chosen_tool in tool_names:
                    result_tool = eval(f"{chosen_tool}('{arg}')")
                    current_message = f"Observation:\n {result_tool}"
                    print(f"chosen tool: {chosen_tool}")
                    print(f"arg: {arg}")
                    print(current_message)
                else:
                    current_message = "Observation: Tool not found"
                continue

            print("----------------------------------------------------------")

            if "Final Answer" in agent_output:
                return agent_output

In [ ]:
client = OpenAI(api_key = os.environ["OPENAI_API_KEY"])
tools = [search, evaluate_expression]
agent = Agent(client, tools)
output = agent.invoke("what is differnce between stock price of microsoft and nvidia in 2024? Which has higher stock price?")
print(f"Final Output-> {output}")

Iteration: 1
Question: what is difference between stock price of microsoft and nvidia in 2024? Which has higher stock price?
Thought: I need to search for the stock prices of Microsoft and Nvidia in 2024 to determine the difference and which one is higher.
Action: search: Microsoft stock price 2024


[('search', 'Microsoft stock price 2024')]
chosen tool: search
arg: Microsoft stock price 2024
Observation:
 Title: Microsoft (MSFT) Stock Forecast and Price Target 2024 - MarketBeat
Snippet: Microsoft - Analysts' Recommendations and Stock Price Forecast (2024) How MarketBeat Calculates Price Target and Consensus Rating. ... According to the research reports of 31 Wall Street equities research analysts, the average twelve-month stock price forecast for Microsoft is $495.68, with a high forecast of $600.00 and a low forecast of $375. ...
----
Title: Microsoft Stock Forecast 2025-2030 | Is MSFT a Good Buy? - Techopedia
Snippet: Their consensus one-year Microsoft (MSFT) stock forecast for 202